In [16]:
import tiktoken
import openai
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers  import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS

In [2]:
loader = PyPDFLoader('./RAG-Practice-PDF-JongHyun.pdf')
pages = loader.load_and_split() # pdf 페이지 단위로 쪼깨서 읽어 들임
print(len(pages))

3


In [6]:
pages[1]

Document(metadata={'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-22T16:01:03+09:00', 'author': 'YechanKim', 'moddate': '2025-05-22T16:01:03+09:00', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}, page_content='박종현 연구원의 업적은 다음과 같음: \n▪ Selected Papers \n▪ International Conference \n▪ Jonghyun Park, Yongmin Ju, Wonil Choi, Hong Kook Kim, and Moongu Jeon, Seamless \nIntegration of 3D LiDAR into ROS-based 2D Navigation Systems, Automation, Robotics & \nCommunications for Industry 4.0/5.0 (ARCI\' 2025), 2025. (Ack: MIT)  \n▪ Y. Kim, J. Park, S. Kim, S. Kim, S. Kim, Y. Ko, J. Oh and M. Jeon, Towards large-scale \nbenchmark dataset for remote sensing object detection on battlefield, ICCE-Asia (IEEE \nInternational Conference on Consumer Electronics-Asia), 2024. (Ack: ADD) \n▪ Domestic Journal \n▪ 김예찬, 박종현, 김수연, 김시현, 김성헌, 고영민, 오중균, 윤동호, 전문구. "위성영상에서의 군 \n표적탐지를 위한 ARMA3 기반의 가상 데이터

In [18]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name="cl100k_base",
)
texts = text_splitter.split_documents(pages)

In [19]:
texts

[Document(metadata={'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-22T16:01:03+09:00', 'author': 'YechanKim', 'moddate': '2025-05-22T16:01:03+09:00', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='영문명 \n- JongHyun Park \n어디 출신? \n- 경기 수원 \n- 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심. \n- 가족 관계로는 부모님과 여동생 1명이 있음. \n병역? \n- 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번\n은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음. \n- 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받\n은 바 있음. \n자격? \n- 인명구조요원, 응급처치법전문과정 자격증 보유 \n어학 연수 경험? \n- 해외 어학 연수 경험 없음 \n- 다만, 유아 시절, ‘영어 유치원’을 다니며 영어 회화 실력을 키워옴 \n학력? \n- 학사 학위는 2021년 2월 경기에 있는 수원대학교에서 취득하였음. 정보통신학부\n를 최우등으로 졸업하였음. \n- 석사 학위는 2023년 8월 광주과학기술원 전기전자컴퓨터공학부에서 취득하였음. \n지도교수는 전문구. \n- 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중. \n \n박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다. \n안녕하십니까? \n박종현 연구원입니다. \n제 소개를 드리도록 하지요.'),
 Document(metadata={'p

In [20]:
len(texts)

3

In [22]:
texts[1].page_content

'박종현 연구원의 업적은 다음과 같음: \n▪ Selected Papers \n▪ International Conference \n▪ Jonghyun Park, Yongmin Ju, Wonil Choi, Hong Kook Kim, and Moongu Jeon, Seamless \nIntegration of 3D LiDAR into ROS-based 2D Navigation Systems, Automation, Robotics & \nCommunications for Industry 4.0/5.0 (ARCI\' 2025), 2025. (Ack: MIT)  \n▪ Y. Kim, J. Park, S. Kim, S. Kim, S. Kim, Y. Ko, J. Oh and M. Jeon, Towards large-scale \nbenchmark dataset for remote sensing object detection on battlefield, ICCE-Asia (IEEE \nInternational Conference on Consumer Electronics-Asia), 2024. (Ack: ADD) \n▪ Domestic Journal \n▪ 김예찬, 박종현, 김수연, 김시현, 김성헌, 고영민, 오중균, 윤동호, 전문구. "위성영상에서의 군 \n표적탐지를 위한 ARMA3 기반의 가상 데이터 생성 연구", Journal of KIISE (2024) \n▪ 한대영, 김예찬, 박종현, 윤동호, 전문구. "A Diffusion-based Data Augmentation Method for \nSAR Object Detection", Journal of KIIT (2024) \n▪ Domestic Conference \n▪ 김수연*, 김예찬*, 박종현, 김성헌, 김기웅, 전문구. "Arma 3로 생성한 군사용 가상 데이터를 \n활용한 실세계 일반 객체탐지 성능 향상에 관한 연구" 한국소프트웨어공학학술대회 (KCSE), \n한국정보과학회x한국정보처리학회 공동주최, 강원 

In [12]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

C:\Users\yechan_amd\AppData\Local\Temp\ipykernel_11820\1537676741.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')


In [23]:
chroma_vector = Chroma.from_documents(texts, model_huggingface)
chroma_retriever = chroma_vector.as_retriever(search_kwargs={'k':2})

질의

In [34]:
docs = chroma_retriever.invoke("박종현 연구원이 실무 담당자로 참여했거나, 참여 중인 과제 목록은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'creator': 'Microsoft® Word Microsoft 365용', 'author': 'YechanKim', 'page': 2, 'source': './RAG-Practice-PDF-JongHyun.pdf', 'creationdate': '2025-05-22T16:01:03+09:00', 'moddate': '2025-05-22T16:01:03+09:00', 'page_label': '3', 'total_pages': 3, 'producer': 'Microsoft® Word Microsoft 365용'}
:
현재 박종현 연구원이 참여 중인 연구 과제는 다음과 같음:  ⚫ 다중 시점의 실세계 환경에서 강건한 밀도추정 및 군중 분석 기술 연구 (후원:  한국연구재단, 실무담당자: 김예찬)  ⚫ 초거대 AI 모델 기반 상황인식 및 의사결심지원 알고리즘 기술 개발 (후원: LIG넥 스원, 실무담당자: 박종현)  ⚫ 새로운 모달리티와 계산 효율적인 대조 학습 연구 (GIST-MIT공동연구, 후원: GIST  자체재원, 실무담당자: 김예찬, 김성헌)    박종현 연구원이 참여하였던 연구 과제는 다음과 같음:  ⚫ 위성영상에서의 트랜스포머 구조 기반 표적탐지/인식 연구 (후원: 국방과학연구 소, 실무담당자: 김예찬)  ⚫ 인공지능 기반 어린이 독서활동 지원 로봇 및 서비스 콘텐츠 개발 (후원: 한국콘 텐츠진흥원, 실무담당자: 박종현, 고영민)    박종현 연구원에 대하여 더 자세한 정보가 알고 싶으시다면,  ⚫ 주소: GIST 전기전자컴퓨터공학부 C동 306호
478
{'source': './RAG-Practice-PDF-JongHyun.pdf', 'creationdate': '2025-05-22T16:01:03+09:00', 'page': 2, 'page_label': '3', 'author': 'YechanKim', 'moddate': '2025-05-22T16:01:03+09:00', 'total_pages': 3, 'creator': 'Microsof

In [33]:
docs = chroma_retriever.invoke("박종현 연구원의 생년월일?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'author': 'YechanKim', 'creationdate': '2025-05-22T16:01:03+09:00', 'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'page': 0, 'page_label': '1', 'total_pages': 3, 'source': './RAG-Practice-PDF-JongHyun.pdf', 'moddate': '2025-05-22T16:01:03+09:00'}
:
영문명  - JongHyun Park  어디 출신?  - 경기 수원  - 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심.  - 가족 관계로는 부모님과 여동생 1명이 있음.  병역?  - 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번 은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음.  - 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받 은 바 있음.  자격?  - 인명구조요원, 응급처치법전문과정 자격증 보유  어학 연수 경험?  - 해외 어학 연수 경험 없음  - 다만, 유아 시절, ‘영어 유치원’을 다니며 영어 회화 실력을 키워옴  학력?  - 학사 학위는 2021년 2월 경기에 있는 수원대학교에서 취득하였음. 정보통신학부 를 최우등으로 졸업하였음.  - 석사 학위는 2023년 8월 광주과학기술원 전기전자컴퓨터공학부에서 취득하였음.  지도교수는 전문구.  - 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중.    박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다.  안녕하십니까?  박종현 연구원입니다.  제 소개를 드리도록 하지요.
621
{'moddate': '2025-05-22T16:01:03+09:00', 'page': 0, 'total_pages': 3, 'creation

In [32]:
docs = chroma_retriever.invoke("박종현 연구원의 군번?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'total_pages': 3, 'source': './RAG-Practice-PDF-JongHyun.pdf', 'creator': 'Microsoft® Word Microsoft 365용', 'author': 'YechanKim', 'page_label': '1', 'moddate': '2025-05-22T16:01:03+09:00', 'creationdate': '2025-05-22T16:01:03+09:00', 'page': 0, 'producer': 'Microsoft® Word Microsoft 365용'}
:
영문명  - JongHyun Park  어디 출신?  - 경기 수원  - 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심.  - 가족 관계로는 부모님과 여동생 1명이 있음.  병역?  - 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번 은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음.  - 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받 은 바 있음.  자격?  - 인명구조요원, 응급처치법전문과정 자격증 보유  어학 연수 경험?  - 해외 어학 연수 경험 없음  - 다만, 유아 시절, ‘영어 유치원’을 다니며 영어 회화 실력을 키워옴  학력?  - 학사 학위는 2021년 2월 경기에 있는 수원대학교에서 취득하였음. 정보통신학부 를 최우등으로 졸업하였음.  - 석사 학위는 2023년 8월 광주과학기술원 전기전자컴퓨터공학부에서 취득하였음.  지도교수는 전문구.  - 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중.    박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다.  안녕하십니까?  박종현 연구원입니다.  제 소개를 드리도록 하지요.
621
{'page': 0, 'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft

In [31]:
docs = chroma_retriever.invoke("박종현 연구원이 석사학위를 취득한 시기는?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-22T16:01:03+09:00', 'author': 'YechanKim', 'page': 0, 'total_pages': 3, 'source': './RAG-Practice-PDF-JongHyun.pdf', 'page_label': '1', 'moddate': '2025-05-22T16:01:03+09:00'}
:
지도교수는 전문구.  - 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중.    박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다.  안녕하십니까?  박종현 연구원입니다.  제 소개를 드리도록 하지요.
126
{'total_pages': 3, 'moddate': '2025-05-22T16:01:03+09:00', 'author': 'YechanKim', 'creationdate': '2025-05-22T16:01:03+09:00', 'page': 0, 'producer': 'Microsoft® Word Microsoft 365용', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'page_label': '1', 'creator': 'Microsoft® Word Microsoft 365용'}
:
영문명  - JongHyun Park  어디 출신?  - 경기 수원  - 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심.  - 가족 관계로는 부모님과 여동생 1명이 있음.  병역?  - 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번 은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음.  - 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받 은 바 있음.  자격?

In [30]:
docs = chroma_retriever.invoke("박종현 연구원이 고소한 직원 이름은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'producer': 'Microsoft® Word Microsoft 365용', 'moddate': '2025-05-22T16:01:03+09:00', 'page_label': '1', 'page': 0, 'total_pages': 3, 'creationdate': '2025-05-22T16:01:03+09:00', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'creator': 'Microsoft® Word Microsoft 365용', 'author': 'YechanKim'}
:
지도교수는 전문구.  - 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중.    박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다.  안녕하십니까?  박종현 연구원입니다.  제 소개를 드리도록 하지요.
126
{'creationdate': '2025-05-22T16:01:03+09:00', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'total_pages': 3, 'page_label': '1', 'author': 'YechanKim', 'creator': 'Microsoft® Word Microsoft 365용', 'producer': 'Microsoft® Word Microsoft 365용', 'page': 0, 'moddate': '2025-05-22T16:01:03+09:00'}
:
영문명  - JongHyun Park  어디 출신?  - 경기 수원  - 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심.  - 가족 관계로는 부모님과 여동생 1명이 있음.  병역?  - 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번 은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음.  - 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받 은 바 있음.  자격?

In [29]:
docs = chroma_retriever.invoke("박종현 연구원의 이메일 주소는?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'moddate': '2025-05-22T16:01:03+09:00', 'page': 0, 'total_pages': 3, 'creator': 'Microsoft® Word Microsoft 365용', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'creationdate': '2025-05-22T16:01:03+09:00', 'author': 'YechanKim', 'producer': 'Microsoft® Word Microsoft 365용', 'page_label': '1'}
:
지도교수는 전문구.  - 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중.    박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다.  안녕하십니까?  박종현 연구원입니다.  제 소개를 드리도록 하지요.
126
{'page': 0, 'source': './RAG-Practice-PDF-JongHyun.pdf', 'author': 'YechanKim', 'page_label': '1', 'total_pages': 3, 'creator': 'Microsoft® Word Microsoft 365용', 'moddate': '2025-05-22T16:01:03+09:00', 'producer': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-22T16:01:03+09:00'}
:
영문명  - JongHyun Park  어디 출신?  - 경기 수원  - 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심.  - 가족 관계로는 부모님과 여동생 1명이 있음.  병역?  - 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번 은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음.  - 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받 은 바 있음.  자격?

In [24]:
docs = chroma_retriever.invoke("박종현 연구원이 주저자로 발표하는 논문들의 각 제목은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'source': './RAG-Practice-PDF-JongHyun.pdf', 'moddate': '2025-05-22T16:01:03+09:00', 'page': 1, 'page_label': '2', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-22T16:01:03+09:00', 'author': 'YechanKim', 'total_pages': 3, 'producer': 'Microsoft® Word Microsoft 365용'}
:
박종현 연구원의 업적은 다음과 같음:  ▪ Selected Papers  ▪ International Conference  ▪ Jonghyun Park, Yongmin Ju, Wonil Choi, Hong Kook Kim, and Moongu Jeon, Seamless  Integration of 3D LiDAR into ROS-based 2D Navigation Systems, Automation, Robotics &  Communications for Industry 4.0/5.0 (ARCI' 2025), 2025. (Ack: MIT)   ▪ Y. Kim, J. Park, S. Kim, S. Kim, S. Kim, Y. Ko, J. Oh and M. Jeon, Towards large-scale  benchmark dataset for remote sensing object detection on battlefield, ICCE-Asia (IEEE
482
{'creator': 'Microsoft® Word Microsoft 365용', 'author': 'YechanKim', 'page_label': '2', 'page': 1, 'total_pages': 3, 'moddate': '2025-05-22T16:01:03+09:00', 'producer': 'Microsoft® Word Microsoft 365용', 'source': './RA

In [25]:
docs = chroma_retriever.invoke("박종현 연구원은 영어 유치원 출신인가?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'total_pages': 3, 'author': 'YechanKim', 'producer': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-05-22T16:01:03+09:00', 'page_label': '1', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'moddate': '2025-05-22T16:01:03+09:00', 'creator': 'Microsoft® Word Microsoft 365용', 'page': 0}
:
영문명  - JongHyun Park  어디 출신?  - 경기 수원  - 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심.  - 가족 관계로는 부모님과 여동생 1명이 있음.  병역?  - 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번 은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음.  - 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받 은 바 있음.  자격?  - 인명구조요원, 응급처치법전문과정 자격증 보유  어학 연수 경험?  - 해외 어학 연수 경험 없음  - 다만, 유아 시절, ‘영어 유치원’을 다니며 영어 회화 실력을 키워옴  학력?  - 학사 학위는 2021년 2월 경기에 있는 수원대학교에서 취득하였음. 정보통신학부 를 최우등으로 졸업하였음.  - 석사 학위는 2023년 8월 광주과학기술원 전기전자컴퓨터공학부에서 취득하였음.  지도교수는 전문구.  - 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중.    박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다.  안녕하십니까?  박종현 연구원입니다.  제 소개를 드리도록 하지요.
621
{'creationdate': '2025-05-22T16:01:03+09:00', 'source': './RAG-Practice-PDF-Jon

In [26]:
docs = chroma_retriever.invoke("박종현 연구원의 여자친구 성명은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("="*50)

{'total_pages': 3, 'page': 0, 'creator': 'Microsoft® Word Microsoft 365용', 'producer': 'Microsoft® Word Microsoft 365용', 'page_label': '1', 'author': 'YechanKim', 'creationdate': '2025-05-22T16:01:03+09:00', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'moddate': '2025-05-22T16:01:03+09:00'}
:
지도교수는 전문구.  - 2023년 9월부터 광주과학기술원 전기전자컴퓨터공학부 박사과정에 재학 중.    박종현 연구원의 여자친구 성명은 ‘김수연’ 입니다.  안녕하십니까?  박종현 연구원입니다.  제 소개를 드리도록 하지요.
126
{'creationdate': '2025-05-22T16:01:03+09:00', 'producer': 'Microsoft® Word Microsoft 365용', 'source': './RAG-Practice-PDF-JongHyun.pdf', 'page': 0, 'total_pages': 3, 'author': 'YechanKim', 'creator': 'Microsoft® Word Microsoft 365용', 'page_label': '1', 'moddate': '2025-05-22T16:01:03+09:00'}
:
영문명  - JongHyun Park  어디 출신?  - 경기 수원  - 1997년 1월 12일에 태어났으며, 부모님은 LG전자 사내연애를 통해 눈이 맞으심.  - 가족 관계로는 부모님과 여동생 1명이 있음.  병역?  - 2016년 1월 18일 입대하여, 2017년 10월 17일 병장으로 해병대를 전역함. 군번 은 16-72000349. 병과는 21/1. 해병대 제1사단 전차대대 소속이었음.  - 해병대 상병 계급일 때, 해병대교육훈련단장으로부터 공수휘장패용증을 수여 받 은 바 있음.  자격?